# Urban Tribes Classification

## Data Preparation

In [1]:
%%time
from tools.datasets.urban_tribes import load_data

# Here we set `validation_split` and `test_split` to very small values to
# accelerate the training process. Don't do this in real runs.
(x_train, y_train), (x_val, y_val), (x_test, y_test) = \
    load_data(images_per_category=2, validation_split=1e-2, test_split=1e-2)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

Using TensorFlow backend.


x_train shape: (22, 224, 224, 3)
y_train shape: (22, 11)
CPU times: user 2.34 s, sys: 314 ms, total: 2.66 s
Wall time: 5.42 s


## Model Building

In [2]:
%%time
from tools import build_transfer_net

model = build_transfer_net(output_dim=11)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

## Training

In [3]:
model.compile(optimizer='adadelta', loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=8, nb_epoch=2,
                    validation_data=(x_val, y_val))

Train on 22 samples, validate on 9 samples
Epoch 1/2
22/22 [==============================] - 32s - loss: 4.1678 - acc: 0.0455 - val_loss: 3.7491 - val_acc: 0.2222
Epoch 2/2
22/22 [==============================] - 35s - loss: 2.6161 - acc: 0.2727 - val_loss: 3.4965 - val_acc: 0.1111


In [4]:
print(model.evaluate(x_test, y_test))
print(model.metrics_names)

11/11 [==============================] - 13s
[2.5210881233215332, 0.18181818723678589]
['loss', 'acc']
